In [31]:
#PARTE 1 - RODAR CKTOOL PARA GERAR METRICAS DOS ARQUIVOS JAVA
import os
import subprocess
from IPython.display import clear_output

# Definir o caminho da pasta
dataset="random"
dir_path = f".\\..\\3_FILES\\{dataset}"

# Criar uma lista vazia para armazenar os nomes dos commits
commit_names = []

for commit in os.listdir(dir_path):
    commit_names.append(commit)

# BEFORE
for i,c in enumerate(commit_names):
    step='before'
    projetosB = f'{dir_path}\\{c}\\{step}'

    command = ["java", "-jar", "tools\\ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar", f'{projetosB}', "true", "0", "false", f'.\\metrics\\{dataset}\\{c}\\{step}\\']
    try:
        clear_output()
        print(f'{step} - {i}/{len(commit_names)}')
        subprocess.run(['mkdir',f'.\\metrics\\{dataset}\\{c}\\{step}'], shell=True)
        subprocess.run(command)
    except Exception as e:
        print(e)

# AFTER
for i,c in enumerate(commit_names):
    step='after'
    projetosB = f'{dir_path}\\{c}\\{step}'

    command = ["java", "-jar", "tools\\ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar", f'{projetosB}', "true", "0", "false", f'.\\metrics\\{dataset}\\{c}\\{step}\\']
    try:
        clear_output()
        print(f'{step} - {i}/{len(commit_names)}')
        subprocess.run(['mkdir',f'.\\metrics\\{dataset}\\{c}\\{step}'], shell=True)
        subprocess.run(command)
    except Exception as e:
        print(e)


# Executar o comando

after - 306/307


In [ ]:
#PARTE 2 - GERAR DATASETS .CSV COM AS METRICAS DOS ARQUIVOS JAVA
import os
import pandas as pd
from IPython.display import clear_output

datasetName=f'random'
datasetNamePath=f'metrics\\{datasetName}'
# Inicializa um dataframe vazio
summaryClass = pd.DataFrame()
summaryMethod = pd.DataFrame()

# Percorre os diretórios dentro de f'{datasetNamePath}'
for i,commit in enumerate (os.listdir(f'{datasetNamePath}')):
    # clear_output()
    print (f'{i}/{len(os.listdir(f"{datasetNamePath}"))}')
    # Verifica se o diretório atual é uma pasta
    if os.path.isdir(os.path.join(f'{datasetNamePath}', commit)):
        # Percorre as pastas 'before' e 'after'
        for folder in ['before', 'after']:
            # Define o caminho para os arquivos CSV
            class_csv_path = os.path.join(f'{datasetNamePath}', commit, folder, 'class.csv')
            method_csv_path = os.path.join(f'{datasetNamePath}', commit, folder, 'method.csv')
            
            # Verifica se os arquivos CSV existem
            if os.path.exists(class_csv_path) and os.path.exists(method_csv_path):
                # Lê os arquivos CSV
                class_df = pd.read_csv(class_csv_path)
                class_df = class_df.add_prefix('classMetric_')
                class_df.insert(0,'project',datasetName,True)
                class_df.insert(0,'commit',commit,True)
                class_df.insert(0,'operation',0,True) #adicionar depois
                class_df.insert(0,'refactored', 0 if folder == 'before' else 1,True)                
                summaryClass = pd.concat([summaryClass, class_df], ignore_index=True, axis=0)

                method_df = pd.read_csv(method_csv_path)
                method_df = method_df.add_prefix('methodMetric_')
                method_df.insert(0,'project',datasetName,True)
                method_df.insert(0,'commit',commit,True)
                method_df.insert(0,'operation',0,True) #adicionar depois
                method_df.insert(0,'refactored', 0 if folder == 'before' else 1,True)                
                summaryMethod = pd.concat([summaryMethod, method_df], ignore_index=True, axis=0)

              
                
# Exibe o dataframe de resumo
summaryClass.to_csv (f'.\\output\\{datasetName}\\datasetClass.csv', index = False, header=True)
summaryMethod.to_csv (f'.\\output\\{datasetName}\\datasetMethod.csv', index = False, header=True)


In [44]:
#PARTE 3 - INCLUIR NO DATASET OS NOMES DAS OPERAÇÕES PUXANDO DO JSON DAS REFATORAÇÕES
import pandas as pd
import json

datasetName='apache'
classMethod='Method'
file_path = f'output/{datasetName}/dataset{classMethod}.csv'
df = pd.read_csv(file_path)


x = open(f'./../2_REFATORAÇÕES/{datasetName}_commit_operations.json', 'r')
myJSON = json.load(x)

df['operation'] = df['operation'].astype('object')
for index, row in df.iterrows():
    # Obter o hash do commit do DataFrame
    commit_hash = row['commit']

    # Encontrar o commit correspondente no JSON
    for commit in myJSON:
        if commit['commit'] == commit_hash:
            df.at[index, 'operation'] = commit['operations']
            break

df.to_csv(f'output/{datasetName}/dataset{classMethod}.csv', index=False)